In [ ]:
%run utils/attention_graph.py
%run utils/mlflow_query.py

In [ ]:
mlflow_helper = MlflowHelper()
mlflow_helper.query_runs(query_metrics=False)

In [ ]:
relevant_mimic_run_df = mlflow_helper.mimic_run_df(include_noise=True, include_refinements=False)
mimic_gram_false_00_run_id = relevant_mimic_run_df[
        (relevant_mimic_run_df['data_tags_noise_type'].fillna('').apply(len) == 0) &   
        (relevant_mimic_run_df['data_tags_model_type'] == 'gram') &
        (relevant_mimic_run_df['data_params_ModelConfigbase_hidden_embeddings_trainable'] == 'False')
].iloc[0].get('info_run_id')
mimic_gram_false_10_run_id = relevant_mimic_run_df[
        (relevant_mimic_run_df['data_tags_noise_type'].fillna('') == 'added0.1_removed0.0_threshold0.0') &   
        (relevant_mimic_run_df['data_tags_model_type'] == 'gram') &
        (relevant_mimic_run_df['data_params_ModelConfigbase_hidden_embeddings_trainable'] == 'False')
].iloc[0].get('info_run_id')
mimic_text_false_00_run_id = relevant_mimic_run_df[
        (relevant_mimic_run_df['data_tags_noise_type'].fillna('').apply(len) == 0) &
        (relevant_mimic_run_df['data_tags_model_type'] == 'text') &
        (relevant_mimic_run_df['data_params_ModelConfigbase_hidden_embeddings_trainable'] == 'False')
].iloc[0].get('info_run_id')
mimic_text_false_10_run_id = relevant_mimic_run_df[
        (relevant_mimic_run_df['data_tags_noise_type'].fillna('') == 'added0.1_removed0.0_threshold0.0') &  
        (relevant_mimic_run_df['data_tags_model_type'] == 'text') &
        (relevant_mimic_run_df['data_params_ModelConfigbase_hidden_embeddings_trainable'] == 'False')
].iloc[0].get('info_run_id')
mimic_causal_false_00_run_id = relevant_mimic_run_df[
        (relevant_mimic_run_df['data_tags_noise_type'].fillna('').apply(len) == 0) &
        (relevant_mimic_run_df['data_tags_model_type'] == 'causal') &
        (relevant_mimic_run_df['data_params_ModelConfigbase_hidden_embeddings_trainable'] == 'False')
].iloc[0].get('info_run_id')
mimic_causal_false_10_run_id = relevant_mimic_run_df[
        (relevant_mimic_run_df['data_tags_noise_type'].fillna('') == 'added0.1_removed0.0_threshold0.0') & 
        (relevant_mimic_run_df['data_tags_model_type'] == 'causal') &
        (relevant_mimic_run_df['data_params_ModelConfigbase_hidden_embeddings_trainable'] == 'False')
].iloc[0].get('info_run_id')
print('Gram', mimic_gram_false_00_run_id, 'Text', mimic_text_false_00_run_id, 'Causal', mimic_causal_false_00_run_id)
print('NOISE 10%: Gram', mimic_gram_false_10_run_id, 'Text', mimic_text_false_10_run_id, 'Causal', mimic_causal_false_10_run_id)

In [ ]:
feature_node_mapping = create_graph_visualization(
    run_id=mimic_gram_false_00_run_id, 
    local_mlflow_dir=mlflow_helper.local_mlflow_dir,
    threshold=0.2, 
    run_name='mimic_gram_false_00', 
    use_node_mapping=False)

In [ ]:
colored_connections, feature_node_mapping = create_graph_visualization_reference(
    run_id=mimic_gram_false_10_run_id, 
    reference_run_id=mimic_gram_false_00_run_id, 
    local_mlflow_dir=mlflow_helper.local_mlflow_dir,
    threshold=0.2, 
    run_name='mimic_gram_false_00', 
    use_node_mapping=False)